In [1]:
import numpy as np

In [2]:
ZOMBIE = "💀"
CAR = "🚘"
BUNKER = "🏠"
EMPTY = "⬜"

In [3]:
# DON'T CHANGE ANYTHING IN THE GRID AS IT IS COUPLED WITH ENVIRONMENT DYNAMICS
# IF YOU WANT TO DO ANY MODIFICATIONS, CONSIDER ADDING THEM TO ENV. DYNAMICS TOO!
grid = [
    [BUNKER, EMPTY, ZOMBIE],
    [ZOMBIE, EMPTY, EMPTY],
    [EMPTY, ZOMBIE, CAR]
]
NUM_STATES = 9

In [4]:
for row in grid:
    print('   '.join(row), end="\n\n")

🏠   ⬜   💀

💀   ⬜   ⬜

⬜   💀   🚘



In [5]:
# 0 -> Right
# 1 -> Up
# 2 -> Left
# 3 -> Down
idx_action = ["Right", "Up", "Left", "Down"]
NUM_ACTIONS = 4
actions = np.arange(NUM_ACTIONS)
actions

array([0, 1, 2, 3])

In [6]:
# SETTING ENVIRONMENT DYNAMICS

# ------------------------------
# REACHING BUNKER: +10
# Getting to ZOMBIE: -50
# TRYING TO GET OUT OF BOUNDS: -5
# NORMAL DRIVING: -1
# ------------------------------

def get_state_and_reward(current_state, action):
    action = idx_action[action]
    if current_state == 8:
        match action:
            case "Right":
                return 8, -5, False
            case "Up":
                return 5, -1, False
            case "Left":
                return 8, -50, True
            case "Down":
                return 8, -5, False
    if current_state == 5:
        match action:
            case "Right":
                return 5, -5, False
            case "Up":
                return 5, -50, True
            case "Left":
                return 4, -1, False
            case "Down":
                return 8, -1, False
    if current_state == 4:
        match action:
            case "Right":
                return 5, -1, False
            case "Up":
                return 1, -1, False
            case "Left":
                return 4, -50, True
            case "Down":
                return 4, -50, True
    if current_state == 1:
        match action:
            case "Right":
                return 1, -50, True
            case "Up":
                return 1, -5, False
            case "Left":
                return 0, 10, True
            case "Down":
                return 4, -1, False

In [7]:
Q_values = np.zeros((NUM_STATES, NUM_ACTIONS))
# Monte Carlo
epsilon = 0.1
alpha = 0.1
current_state = 8
# Will give optimal policy even in 100 iterations <3
for i in range(100000):
    greedy_action = np.argmax(Q_values[current_state])
    action = greedy_action
    if np.random.rand() < epsilon:
        # Explore
        while action == greedy_action:
            action = np.random.randint(0, NUM_ACTIONS)
    new_state, reward, end = get_state_and_reward(current_state, action)
    Q_values[current_state][action] += alpha*(reward + np.max(Q_values[new_state]) - Q_values[current_state][action])
    if end:
        current_state = 8
    else:
        current_state = new_state

In [8]:
Q_values

array([[  0.,   0.,   0.,   0.],
       [-40.,   5.,  10.,   8.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  7.,   9., -41., -41.],
       [  3., -42.,   8.,   6.],
       [  0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.],
       [  2.,   7., -43.,   2.]])

In [9]:
# Optimal Policy
pi = np.argmax(Q_values, axis=1)
pi

array([0, 2, 0, 0, 1, 2, 0, 0, 1])

In [10]:
for row in grid:
    print('   '.join(row), end="\n\n")
print("\n")
# Following optimal policy
current_state = 8
end = False
while not end:
    action = pi[current_state]
    print(f"Took action: {idx_action[action]}")
    current_state, reward, end = get_state_and_reward(current_state, action)

🏠   ⬜   💀

💀   ⬜   ⬜

⬜   💀   🚘



Took action: Up
Took action: Left
Took action: Up
Took action: Left
